In [1]:
import pandas as pd
import pyarrow.parquet as pq

cols = ['customer_id', 'event_type', 'cost', 'sales', 'timestamp_utc']
parquet_file = pq.ParquetFile('data/customer_journey.parquet')

# Étape 1 : On doit d'abord identifier la 1ère exposition par utilisateur sur TOUT le fichier
# car la 1ère expo peut être dans le batch 1 et une vente dans le batch 10.
print("Recherche des dates de première exposition...")
user_first_expo = {}

for batch in parquet_file.iter_batches(batch_size=500_000, columns=['customer_id', 'event_type', 'timestamp_utc']):
    chunk = batch.to_pandas()
    exposures = chunk[chunk['event_type'] == 'exposure']
    if not exposures.empty:
        # On trouve le min par batch
        batch_min = exposures.groupby('customer_id')['timestamp_utc'].min()
        # On met à jour le dictionnaire global si le nouveau min est plus ancien
        for cid, t in batch_min.items():
            if cid not in user_first_expo or t < user_first_expo[cid]:
                user_first_expo[cid] = t

# On transforme le dictionnaire en Series pour une jointure rapide
df_first_expo = pd.Series(user_first_expo, name='first_exposure_ts')

# Étape 2 : Deuxième passage pour calculer les ventes (Total vs Post-Expo)
print("Calcul des revenus...")
aggregated_data = []

for batch in parquet_file.iter_batches(batch_size=500_000, columns=cols):
    chunk = batch.to_pandas()
    
    # On ramène la date de 1ère expo pour chaque ligne du chunk
    chunk = chunk.join(df_first_expo, on='customer_id')
    
    # Ventes totales
    chunk['sales_total'] = chunk['sales'].fillna(0)
    
    # Ventes APRES exposition (uniquement si timestamp_utc > first_exposure_ts)
    chunk['sales_post_expo'] = 0.0
    mask_post = (chunk['event_type'] != 'exposure') & \
                (chunk['sales'] > 0) & \
                (chunk['timestamp_utc'] > chunk['first_exposure_ts'])
    chunk.loc[mask_post, 'sales_post_expo'] = chunk.loc[mask_post, 'sales']

    summary = chunk.groupby('customer_id').agg({
        'sales_total': 'sum',
        'sales_post_expo': 'sum',
        'cost': 'sum',
        'event_type': lambda x: (x == 'exposure').sum()
    })
    aggregated_data.append(summary)

# Fusion finale
user_metrics = pd.concat(aggregated_data).groupby(level=0).sum().reset_index()
user_metrics.rename(columns={'event_type': 'nb_impressions', 'cost': 'budget_media_raw'}, inplace=True)
user_metrics['budget_media'] = user_metrics['budget_media_raw'] / 1000

# Résultats
user_metrics = user_metrics[['customer_id', 'nb_impressions', 'budget_media', 'sales_total', 'sales_post_expo']]

Recherche des dates de première exposition...
Calcul des revenus...


In [ ]:
# Calcul des totaux globaux
total_sales_gbp = user_metrics['sales_total'].sum()
total_sales_post_expo_gbp = user_metrics['sales_post_expo'].sum()
total_budget_gbp = user_metrics['budget_media'].sum()

# Calcul des ratios (sans unité monétaire)
roas = total_sales_gbp / total_budget_gbp if total_budget_gbp > 0 else 0
proas = total_sales_post_expo_gbp / total_budget_gbp if total_budget_gbp > 0 else 0

print(f"--- Rapport de Performance (Devise : $) ---")
print(f"Total Budget investi : ${total_budget_gbp:,.2f}")
print(f"Total Ventes : ${total_sales_gbp:,.2f}")
print(f"Total Ventes (Post-Expo) : ${total_sales_post_expo_gbp:,.2f}")
print(f"-------------------------------------------")
print(f"ROAS Global : {roas:.2f}x")  # On ajoute souvent un 'x' pour marquer le multiplicateur
print(f"ROAS Post-Exposition : {proas:.2f}x")

--- Rapport de Performance (Devise : $) ---
Total Budget investi : $19,360,334.77
Total Ventes (Post-Expo) : $22,274,509.64
-------------------------------------------
ROAS Global : 3.16x
ROAS Post-Exposition : 1.15x
